In [134]:
import pandas as pd
import requests
import pickle

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [135]:
spotify_client_id = 'e0e51ec7feeb45b89b165ae1007c535d'
spotify_client_secret = 'cf444557e7ad446ebd5c2d4cc6a781cf'

client_credentials_manager = SpotifyClientCredentials(spotify_client_id, spotify_client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [136]:
def dropDictKeys(dict, keys):
    for key in keys:
        try:
            del dict[key]
        except Exception as e:
            pass
    return dict

def keepDictKeys(dict, keys):
    new_dict = {}
    if dict:
        for key in keys:
            new_dict[key] = dict[key]
        return new_dict
    return {}

In [137]:
artist_lyrics = pickle.load(open('../collection/2 Chainz.pkl', 'rb'))

In [138]:
artist_lyrics.keys()

dict_keys(['header_image_url', 'id', 'image_url', 'is_verified', 'name', 'url', 'iq', 'albums'])

In [139]:
artist_lyrics['albums'][0]['tracks'][0].keys()

dict_keys(['number', '_type', 'full_title', 'header_image_thumbnail_url', 'header_image_url', 'id', 'lyrics_state', 'song_art_image_thumbnail_url', 'song_art_image_url', 'stats', 'title', 'title_with_featured', 'url', 'lyrics', 'description', 'embed_content', 'featured_video', 'recording_location', 'apple_music_id', 'apple_music_player_url', 'release_date', 'release_date_for_display', 'featured_artists', 'producer_artists', 'writer_artists', 'custom_performances', 'song_relationships'])

In [140]:
for artist in [artist_lyrics]:
    artist_match = sp.search(artist['name'], type='artist')
    artist_match = artist_match['artists']['items'][0]

    keep = ['followers', 'popularity', 'genres'] #'id', 'name', 
    drop = ['external_urls', 'type', 'href', 'uri', 'images']

    artist = {**artist, **keepDictKeys(artist_match, keep)}
    artist['followers'] = artist['followers']['total']
    for i, album in enumerate(artist['albums']):

        album_match = sp.search(album['name'].split('-')[0], type='album')['albums']['items']
        if album_match:
            album_id = album_match[0]['id']
        else:
            album_id = 'None'
        album_match = sp.albums([album_id])['albums'][0]

        keep = ['genres', 'popularity', 'release_date'] # 'name', 'id', 'type', 'tracks'
        album_match = keepDictKeys(album_match, keep) or {'genres': None, 'popularity': None, 'release_date': None}
        artist['albums'][i] = {**album, **album_match}
        
        
        for j, track in enumerate(album['tracks']):
            
            query = f"artist:{artist['name']} track:{track['title']}"
            track_match = sp.search(query, type='track')['tracks']['items']
        

            if track_match:
                track_id = track_match[0]['id']
            else:
                track_id = 'None'
            track_match = sp.tracks([track_id])['tracks'][0]

            audio_features = sp.audio_features(track_id)[0]
            audio_features = audio_features if audio_features else {}
            drop = ['id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature', 'type']
            track['audio_features'] = dropDictKeys(audio_features, drop)

            keep =['popularity', 'duration_ms'] #'id', 'name'
            track_match = keepDictKeys(track_match, keep) or {'popularity': None, 'duration_ms': None, 'audio_features': None}
            artist['albums'][i]['tracks'][j] = {**track, **track_match}

In [142]:
artist['albums'][0]['tracks'][0].keys()

dict_keys(['number', '_type', 'full_title', 'header_image_thumbnail_url', 'header_image_url', 'id', 'lyrics_state', 'song_art_image_thumbnail_url', 'song_art_image_url', 'stats', 'title', 'title_with_featured', 'url', 'lyrics', 'description', 'embed_content', 'featured_video', 'recording_location', 'apple_music_id', 'apple_music_player_url', 'release_date', 'release_date_for_display', 'featured_artists', 'producer_artists', 'writer_artists', 'custom_performances', 'song_relationships', 'audio_features', 'popularity', 'duration_ms'])

In [76]:
# for artist in [artist_lyrics]:
#     artist_match = sp.search(artist['name'], type='artist')
#     artist_match = artist_match['artists']['items'][0]

#     keep = ['followers', 'popularity', 'genres'] #'id', 'name', 
#     drop = ['external_urls', 'type', 'href', 'uri', 'images']

#     artist = {**artist, **keepDictKeys(artist_match, keep)}
#     artist['followers'] = artist['followers']['total']

#     # ALBUMS
#     album_ids = []
#     for _i, album in enumerate(artist['albums']):

#         album_match = sp.search(album['name'].split('-')[0], type='album')['albums']['items']
#         if album_match:
#             album_id = album_match[0]['id']
#         else:
#             album_id = 'None'
#         album_ids.append(album_id)

#     albums = []
#     fives, remainder = len(album_ids) // 5, len(album_ids) % 5
#     m, n = 0, 5
#     for _i in range(fives + 1):
#         if album_ids[m:n]:
#             albums = [*albums, *sp.albums(album_ids[m:n])['albums']]
#             m += 5; n += 5
#     albums = [album if album else {'genres': None, 'popularity': None, 'release_date': None} for album in albums]

#     for i, album in enumerate(artist['albums']):
#         keep = ['genres', 'popularity', 'release_date'] # 'name', 'id', 'type', 'tracks'
#         artist['albums'][i] = {**album, **keepDictKeys(albums[i], keep)}
    
#     # TRACKS
#     track_ids = []
#     for _j, track in enumerate(album['tracks']):
        
#         query = f"artist:{artist['name']} track:{track['title']}"
#         track_match = sp.search(query, type='track')['tracks']['items']
    
#         if track_match:
#             track_id = track_match[0]['id']
#         else:
#             track_id = 'None'
#         track_ids.append(track_id)

#     tracks = []
#     fives, remainder = len(track_ids) // 5, len(track_ids) % 5
#     m, n = 0, 5
#     for _j in range(fives + 1):
#         if track_ids[m:n]:
#             tracks = [*tracks, *sp.tracks(track_ids[m:n])['tracks']]
#             m += 5; n += 5
#     tracks = [track if track else {'popularity': None, 'duration_ms': None, 'audio_features': None} for track in tracks]

#     audio_features = sp.audio_features(track_ids)
#     audio_features = [af if af else {} for af in audio_features]

#     for j, track in enumerate(album['tracks']):
        
#         drop = ['id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature', 'type']
#         track['audio_features'] = dropDictKeys(audio_features[j], drop)

#         keep =['popularity', 'duration_ms'] #'id', 'name'
        
#         artist['albums'][i]['tracks'][j] = {**track, **keepDictKeys(tracks[j], keep)}